### Environment

In [ ]:
#need to install this for clinical coding
# !pip install scispacy
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz

In [2]:
# import  packages you need
import numpy as np
import logging
import os
from typing import Callable, Dict, List, Optional, Tuple
import csv
import json, time
from collections import defaultdict
from datetime import datetime
from itertools import combinations, islice
import pickle
import pandas as pd
import torch
import pyhealth
from pyhealth.medcode import InnerMap
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import scispacy #from above
import spacy #from above
import random

### model 0 Data


In [9]:
notedir = './sample_data/mimic-iv/note/'
hospdir = './sample_data/mimic-iv/hosp/'
diags = pd.read_csv(hospdir + 'diagnoses_icd.csv.gz')
d_notes = pd.read_csv(notedir + 'discharge.csv.gz')
r_notes = pd.read_csv(notedir + 'radiology.csv.gz')
r_notes = r_notes.dropna(subset=['hadm_id'])
# Group radiology text into per visit 
radiology_grouped = r_notes.groupby(['subject_id','hadm_id'])['text'].apply(' '.join).reset_index()

diags10 = diags[diags.icd_version == 10]
# pancreas = diags10[diags10['icd_code'].astype(str).str.contains('C25')]
#encode all diagnoses into either times where pancreatic cancer identified or not
diags10.loc[:, 'contains_C25'] = (diags10.loc[:,'icd_code'].str.contains('C25')).astype(int)
diags_encoded = diags10.groupby(['subject_id', 'hadm_id']).agg({'contains_C25': 'max'}).reset_index()
diags_encoded.rename(columns={'contains_C25': 'icd_code'}, inplace=True)
diags_encoded['icd_code'] = diags_encoded['icd_code'].apply(lambda x: 0 if x == 0 else 1)

#just the first 512 of each if true
small_df = False

if small_df:
    merged_df = pd.merge(radiology_grouped.head(512), d_notes.head(512), on=['subject_id', 'hadm_id'], how='right')
    merged_df['text'] = merged_df['text_x'] + ' ' + merged_df['text_y']
    merged_df = merged_df.drop(columns=['text_x', 'text_y']).dropna(subset=['text'])
    merged_df = pd.merge(diags_encoded.head(512), merged_df, on=['subject_id', 'hadm_id'], how='inner').drop(columns=['note_id','note_seq','charttime','storetime', 'note_type'])
else:
    merged_df = pd.merge(radiology_grouped, d_notes, on=['subject_id', 'hadm_id'], how='right')
    merged_df['text'] = merged_df['text_x'] + ' ' + merged_df['text_y']
    merged_df = merged_df.drop(columns=['text_x', 'text_y']).dropna(subset=['text'])
    merged_df = pd.merge(diags_encoded, merged_df, on=['subject_id', 'hadm_id'], how='inner').drop(columns=['note_id','note_seq','charttime','storetime', 'note_type'])

print(merged_df.shape)
merged_df.head()

C:\Users\Franlk\AppData\Local\Temp\ipykernel_19608\2097346371.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diags10.loc[:, 'contains_C25'] = (diags10.loc[:,'icd_code'].str.contains('C25')).astype(int)


(97301, 4)


,subject_id,hadm_id,icd_code,text
0,10000117,22927623,0,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATIO...
1,10000117,27988844,0,"EXAMINATION: Left hip radiographs, two views,..."
2,10000980,20897796,0,EXAMINATION: BILAT LOWER EXT VEINS\n\nINDICAT...
3,10000980,25911675,0,EXAMINATION: Chest radiograph.\n\nINDICATION:...
4,10000980,29659838,0,INDICATION: ___ with c/o SOB // ? PNA or CH...


#### Model 0 Tokenizer

Because the clinical notes can be very long (sometimes up to 20,000 words) the tokenizer can take hours to run. We have selected a smaller subset of the data that will be pre processed for model 0. The number of rows with a record containing pancreatic cancer is around 1500, and the number containing other diseases is around 4500

In [10]:
# Increase ratio to get more non-pancreatic cancer data
ratio = .05

ones = merged_df.index[(merged_df.icd_code == 1)].tolist()
zeros = merged_df.index[(merged_df.icd_code == 0)].tolist()

N = int(ratio * len(zeros))
subset_zeros = random.sample(zeros, N)
indices = ones + subset_zeros
subset_df = merged_df.iloc[indices]
subset_df.head()

,subject_id,hadm_id,icd_code,text
69,10006029,20850584,1,"EXAMINATION: Chest radiograph, portable AP up..."
70,10006029,25426298,1,INDICATION: ___ year old man with metastatic ...
74,10006431,25589898,1,EXAMINATION: CTA ABD AND PELVIS\n\nINDICATION...
75,10006431,27715811,1,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
76,10006431,28771670,1,"INDICATION: ___ with upper abdominal pain, hi..."


The process to tokenise first begins by removing some common fluff that exists in the text records. Then run the text through scispacy- a biomedical text processing library which will remove any extranenous filler words and return a more simplifed sequence. Then, using the predefined tokenizer from [Bio_ClinicalBERT](https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT) the text in each row is tokenized and either padded or in a few cases truncated to a max length of 8192. (This whole process takes about 2 seconds per row)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
nlp = spacy.load("en_core_sci_sm")
def process_row(row):
    text = row['text']
    text = text.replace('\n', '')
    text = text.replace('___','')
    text = text.replace('Name:                   Unit No:    Admission Date:                Discharge Date:    Date of Birth:', '')
    doc = nlp(text)
    ents = doc.ents
    t = ''
    for s in ents:
        t += s.text
        t += ' '
    
    t = t.replace('=','')
    tok = tokenizer(t, return_tensors="pt", max_length=8192, truncation=True, padding='max_length')
    return tok

tts = subset_df.apply(process_row, axis=1)

inps = torch.stack([tok['input_ids'] for tok in tts])
inps = inps.squeeze(1).tolist()
masks = torch.stack([tok['attention_mask'] for tok in tts])
masks = masks.squeeze(1).tolist()
subset_df['encoded_text'] = inps
subset_df['masks'] = masks
subset_df.drop(columns='text').to_pickle('./subset_df.pkl')


#### the tokenised pickle data can be [downloaded here](https://drive.google.com/file/d/1OIG8JaMfg7x5Cl9Losf8a_SJ7smxDic0/view?usp=drive_link) and loaded like this in lieu of running more pre processing:


In [5]:
df = pd.read_pickle('subset_df.pkl')
print(df.shape)
df.head()

(6039, 5)


,subject_id,hadm_id,icd_code,encoded_text,masks
69,10006029,20850584,1,"[101, 8179, 2229, 2070, 15241, 170, 1643, 1275...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
70,10006029,25426298,1,"[101, 12754, 1299, 27154, 27372, 22572, 5326, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
74,10006431,25589898,1,"[101, 8179, 172, 1777, 170, 1830, 1181, 185, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
75,10006431,27715811,1,"[101, 8179, 185, 1161, 5837, 7563, 13335, 2566...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
76,10006431,28771670,1,"[101, 12754, 3105, 24716, 2489, 1607, 13316, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


### Model for model 0 

The base model used for the pretrained model comes from the [Bio_ClinicalBERT](https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT). This model was trained on ~880 million words from discharge summary notes from the [mimic III dataset](https://mimic.mit.edu/). Their base model can be loaded as follows: 


In [ ]:
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)

### Model 0 training/fine tuning

Following the [original paper](https://www.nature.com/articles/s41467-023-43715-z), the training and testing data was split into the ratio of 95-5 due to memory issues the batch size was reduced to 16 rather than using the original of 48.

the model can only handle a max_length of 512, therefore chunking was implemented for better training capture. The model was trained over 5 epochs on a google collab T4 GPU with 15 GB of memory with each epoch taking roughly 12 minutes for a total training time of around an hour.


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_ids = df['encoded_text'].tolist()
attention_masks = df['masks'].tolist()
labels = df['icd_code'].tolist()

batch_size = 16

ids = torch.tensor(input_ids).to(device)
masks = torch.tensor(attention_masks).to(device)
labels = torch.tensor(labels).to(device)

dataset = TensorDataset(ids, masks, labels)

train_size = int(0.95 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

max_length = 512
chunk_size = 512


In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        input_ids_batch, attention_masks_batch, labels_batch = batch
        optimizer.zero_grad()

        # chunk
        chunk_logits = []
        for i in range(0, len(input_ids_batch), chunk_size):
            input_ids_chunk = input_ids_batch[:,i:i+chunk_size].to(device)
            attention_masks_chunk = attention_masks_batch[:,i:i+chunk_size].to(device)
            logits = model(input_ids=input_ids_chunk, attention_mask=attention_masks_chunk).logits
            chunk_logits.append(logits)

        logits = torch.mean(torch.stack(chunk_logits), dim=0)

        loss = criterion(logits, labels_batch)
        loss.backward()
        optimizer.step()

#### This model was saved and [upload here](https://drive.google.com/file/d/1jehT4uyz6465kX5Ho-ddn1qikzyso5vV/view?usp=drive_link) This tuned model can be loaded as follows:

In [18]:
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)
path = './5_tuned_model.pth'
model.load_state_dict(torch.load(path, map_location=torch.device(device)))


### model 0 evaluation

The model was then tested on the remaining 5% of the dataset using the same chunking method as was used in training and yielded the following scores:

Accuracy: 0.9511589403973509

Precision: 0.907258064516129

Recall: 0.8620689655172413

F1-score: 0.8840864440078585


In [20]:
# Evaluation loop
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)
path = './5_tuned_model.pth'
model.load_state_dict(torch.load(path, map_location=torch.device(device)))

model.eval()
predictions = []
true_labels = []

max_length = 512
chunk_size = 512

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_batch, attention_masks_batch, labels_batch = batch
                # chunk
        chunk_logits = []
        for i in range(0, len(input_ids_batch), chunk_size):
            input_ids_chunk = input_ids_batch[:,i:i+chunk_size]
            attention_masks_chunk = attention_masks_batch[:,i:i+chunk_size]
            logits = model(input_ids=input_ids_chunk, attention_mask=attention_masks_chunk).logits
            chunk_logits.append(logits)
        logits = torch.mean(torch.stack(chunk_logits), dim=0)
        _, predicted = torch.max(logits, dim=1)
        predictions.extend(predicted.tolist())
        true_labels.extend(labels_batch.tolist())


In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.9511589403973509
Precision: 0.907258064516129
Recall: 0.8620689655172413
F1-score: 0.8840864440078585
